In [18]:
import pandas as pd
import json
import csv
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import math


# Detailed Listings Formatting

In [61]:
#These functions help transform the ammenities column from a list-like string to a list that MultiLabelBinarizer can work on
def strip_split(x):
    return x.strip("[]' ").split(",")

def striper(x):
    return x.strip("[]' ")

def formater(x):
    alist = list(map(strip_split, x))
    blist = []
    for i in range(0,len(alist)):
        blist.append(list(map(striper, alist[i])))
    return blist

In [62]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
 detailed_listings_back.columns = detailed_listings_back.columns.str.replace("content.","")
detailed_listings_back.city = detailed_listings_back.city.str.strip()

if 'amenities' in detailed_listings_back.columns:
    s = detailed_listings_back['amenities']
    mlb = MultiLabelBinarizer()
    ammenities_df = pd.DataFrame(mlb.fit_transform(formater(s)),columns=mlb.classes_, index=detailed_listings_back.index)
    if "Wifi" not in detailed_listings_back.columns:
        detailed_listings_back = detailed_listings_back.join(ammenities_df)
    detailed_listings_back.drop("amenities", axis = 1)

InitialMissingCells = np.sum(detailed_listings_back.isnull().sum()) 
InitialRowCount = detailed_listings_back.shape[0]
InitialColumnCount = detailed_listings_back.shape[1]
print("Dataset shape is " + str(detailed_listings_back.shape))
print(str(InitialMissingCells) + " null cells in initial dataset")
print(str(InitialMissingCells/InitialRowCount/InitialColumnCount) + "% of dataset is null")
ammenities_csv_data = ammenities_df.to_csv('l0_amenities.csv', index = False) 
#yes, my spelling isn't uniformly perfect


Dataset shape is (1497, 283)
25057 null cells in initial dataset
0.05914538145785092% of dataset is null


# Column Drops

In [63]:
#Drops for relevance: status, id,picture_url, thumbnail_url, medium_url,xl_picture_url, country, country_code,
#market, state, bed_type (identical to bed_type_category), amenities (unnescesary after binarization), 
#native_price(identical to price), price_formatted(formatted as string), security_deposit_formatted (formatted as string)
#security_price_native(identical to security_deposit_native), smart_location(identical to city), address (identical to city)
#loc.coordinates(identical to lat,lng), localized_city(identical to city), airbnb_id(each one is unique), mashvisor_id(each is unique)
#host_id(identical to user_id), listing_monthly_price_native (identical to monthly_price_native),
#listing_weekly_price_native (identical to weekly_price_native), extras_price_native (identical to cleaning_fee_native),
#listing_cleaning_fee_native(identical to cleaning_fee_native), language(identical to locale), Other (Not meaningful),
#listing_price_for_extra_person_native, min_nights_input_value, max_nights_input_value
#cancel_policy

#Drops for simplicity: name, public_address, house_rules, description, summary, space, access, interaction, transit,
#neighborhood_overview, user_id, created_at, updated_at

#Drops where 80% or better of data is missing:is_business_travel_ready, weekly_price_native, monthly_price_native 

detailed_drops = ["status","id","price_native", "price_formatted", "picture_url", "thumbnail_url", "name",
                  "medium_url", "xl_picture_url", "security_deposit_formatted", "smart_location", "address",
                  "country","country_code","security_price_native", "is_business_travel_ready",
                  "listing_monthly_price_native","listing_weekly_price_native", "extras_price_native",
                  "listing_cleaning_fee_native","language", "Other",
                 "market", "state","currency_symbol_right", "license","square_feet", "map_image_url",
                  "has_viewed_terms","has_viewed_cleaning", "has_viewed_ib_perf_dashboard_panel","special_offer",
                  "toto_opt_in","wireless_info", "amenities", "bed_type", "public_address", "house_rules",
                  "description", "summary", "space", "access", "interaction", "loc.coordinates","airbnb_id",
                  "mashvisor_id", "localized_city", "public_address", "house_rules", "description", "summary",
                 "space", "access", "interaction", "transit", "neighborhood_overview", "host_id", 
                  "weekly_price_native", "monthly_price_native", "listing_price_for_extra_person_native", "min_nights_input_value",
                 "max_nights_input_value", 'cancel_policy', 'created_at', 'updated_at']
for detailed_drop in detailed_drops:
    if detailed_drop in detailed_listings_back.columns:
        detailed_listings_back = detailed_listings_back.drop(detailed_drop, axis = 1)

for i in detailed_listings_back.columns:
    if len(detailed_listings_back[i].unique()) == 1:
        if np.any(detailed_listings_back[i].isnull()) == False:
            detailed_listings_back = detailed_listings_back.drop(i, axis = 1)
        
detailed_listings_back

print("Dataset shape is " + str(detailed_listings_back.shape))
print(str(InitialMissingCells) + " null cells in initial dataset")
print(str(np.sum(detailed_listings_back.isnull().sum()))+ " in current dataset")
print(str(np.sum(detailed_listings_back.isnull().sum())/detailed_listings_back.shape[0]/detailed_listings_back.shape[1]) + "% of dataset is null")

Dataset shape is (1497, 212)
25057 null cells in initial dataset
2264 in current dataset
0.007133764384114141% of dataset is null


# Variance Filtering

In [64]:
lVarDrops = detailed_listings_back.var() < (math.log(detailed_listings_back.shape[0])/1497)
vdrops = lVarDrops.iloc[lVarDrops.values == True].keys()
for vdrop in vdrops:
    if vdrop in detailed_listings_back.columns:
        detailed_listings_back = detailed_listings_back.drop(vdrop, axis = 1)

for vdrop in vdrops:
    if vdrop in ammenities_df.columns:
        ammenities_df = ammenities_df.drop(vdrop, axis = 1)

amdrop_csv_data = ammenities_df.to_csv('l1_amenities.csv', index = False) 
print("Ammenities dropped for low variance are:")
print(lVarDrops.iloc[lVarDrops.values == True].keys())
print("Dataset shape is " + str(detailed_listings_back.shape))
print(str(InitialMissingCells) + " null cells in initial dataset")
print(str(np.sum(detailed_listings_back.isnull().sum()))+ " in current dataset")
print(str(np.sum(detailed_listings_back.isnull().sum())/detailed_listings_back.shape[0]/detailed_listings_back.shape[1]) + "% of dataset is null")


Ammenities dropped for low variance are:
Index(['lat', 'lng', 'has_agreed_to_legal_terms', 'weekly_price_factor',
       'Amazon Echo', 'Baby monitor', 'Balcony', 'Bread maker',
       'Convection oven', 'DVD player', 'Day bed', 'En suite bathroom',
       'Espresso machine', 'Fire pit', 'Firm mattress',
       'Fixed grab bars for shower', 'Formal dining area', 'HBO GO',
       'Hand Sanitiser', 'Handheld shower head', 'Heat lamps', 'Heated floors',
       'Jetted tub', 'Kitchenette', 'Mudroom', 'Murphy bed',
       'Nespresso machine', 'Other pet(s)', 'Outdoor parking', 'Piano',
       'Pillow-top mattress', 'Playground', 'Printer', 'Rain shower',
       'Record player', 'Rice Maker', 'Roll-in shower', 'Soaking tub',
       'Sound system', 'Table corner guards', 'Terrace', 'Trash compacter',
       'Walk-in shower', 'Wide doorway to guest bathroom', 'Wide entrance',
       'Wide entryway'],
      dtype='object')
Dataset shape is (1497, 166)
25057 null cells in initial dataset
1591 in

# Proportion of Missing Rows in Each Column (NaN/NA)

In [65]:
print(detailed_listings_back.isnull().sum()/detailed_listings_back.shape[0])

city                                          0.000668
user_id                                       0.000000
price                                         0.000000
instant_bookable                              0.000000
bedrooms                                      0.000668
beds                                          0.000000
bathrooms                                     0.000000
min_nights                                    0.000000
neighborhood                                  0.000000
person_capacity                               0.000000
zipcode                                       0.000000
cancellation_policy                           0.000000
property_type                                 0.000000
reviews_count                                 0.000000
picture_count                                 0.000000
bed_type_category                             0.000000
require_guest_profile_picture                 0.000000
require_guest_phone_verification              0.000000
check_in_t

# Data Cleaning

In [66]:
#Normalizing City Names
detailed_listings_back['city'] = detailed_listings_back['city'].replace(['Passyunk Square, Philadelphia'],'Philadelphia')
detailed_listings_back['city'] = detailed_listings_back['city'].replace(np.nan, "Philedelphia") #Inferred from Neighborhood

#Removing very inconsequential missing values. THIS LINE NEEDS TO BE ACTIVATED BEFORE USE!!!
lmDrops = math.log10(len(detailed_listings_back))/detailed_listings_back.shape[0]
detailed_listings_back.dropna(axis = 0, subset = detailed_listings_back.columns[detailed_listings_back.isnull().sum()/len(detailed_listings_back) <= lmDrops].to_list()).shape

#imputing normal daily price to observations with no weekend rate (confirmed w/ manual website inspection)
detailed_listings_back['listing_weekend_price_native'] = detailed_listings_back['listing_weekend_price_native'].replace(np.nan, detailed_listings_back['price'])

#monthly_price_factor and weekly_price_factor are not listed in the API documentation but appear to correspond with 
#discount percentages on the airbnb website. Imputing 1 where it is missing
detailed_listings_back['monthly_price_factor'] = detailed_listings_back['monthly_price_factor'].replace(np.nan, 1)
if 'weekly_price_factor' in detailed_listings_back.columns:
    detailed_listings_back['weekly_price_factor'] = detailed_listings_back['weekly_price_factor'].replace(np.nan, 1)

#max_nights_input_values has lots ot NA values but no zeroes, so I'm taking that as infinity
#detailed_listings_back['max_nights_input_value'] = detailed_listings_back['max_nights_input_value'].replace(np.nan, float("inf"))

#min_nights_input_values has lots ot NA values but no zeroes, so I'm taking that as 1.
#I don't think AirBnb rents by the hour anyway :)
if 'min_nights_input_value' in detailed_listings_back.columns:
    detailed_listings_back['min_nights_input_value'] = detailed_listings_back['min_nights_input_value'].replace(np.nan, 1)

###NAs in these three seem to be connected. Deleting all rows with NAs in occupancy results in removal of all rows with
###NAs in the other two and only one row with an NA in occupancy has a value in rental income, which was very low

#occupancy has lots ot NA values but no zeroes, so I'm taking that as 0.
detailed_listings_back['occupancy'] = detailed_listings_back['occupancy'].replace(np.nan, 0)

#nights_booked has lots ot NA values but no zeroes, so I'm taking that as 0.
detailed_listings_back['nights_booked'] = detailed_listings_back['nights_booked'].replace(np.nan, 0)

#rental_income has lots ot NA values but no zeroes, so I'm taking that as 0.
detailed_listings_back['rental_income'] = detailed_listings_back['rental_income'].replace(np.nan, 0)

#Imputed creation date to observations that have no update date and creating date/time format cells
if 'updated_at' in detailed_listings_back:
    detailed_listings_back['updated_at'] = detailed_listings_back['updated_at'].replace(np.nan, detailed_listings_back['created_at'])
#    detailed_listings_back['date_updated'] = detailed_listings_back['updated_at'].str.split('T', expand=True)[0]
#    detailed_listings_back['time_updated'] = detailed_listings_back['updated_at'].str.split('T', expand=True)[1]
#    detailed_listings_back['time_updated'] = detailed_listings_back['time_updated'].str.split('.', expand=True)[0]
#    detailed_listings_back.drop(axis = 1, labels = 'updated_at')
    
#if 'created_at' in detailed_listings_back:
#    detailed_listings_back['date_created'] = detailed_listings_back['created_at'].str.split('T', expand=True)[0]
#    detailed_listings_back['time_created'] = detailed_listings_back['created_at'].str.split('T', expand=True)[1]
#    detailed_listings_back['time_created'] = detailed_listings_back['time_created'].str.split('.', expand=True)[0]
#    detailed_listings_back.drop(axis = 1, labels = 'created_at')

#Imputing description_locale to locale where missing
detailed_listings_back['locale'] = detailed_listings_back['locale'].replace(np.nan, detailed_listings_back['description_locale'])

print("Dataset shape is " + str(detailed_listings_back.shape))
print(str(InitialMissingCells) + " null cells in initial dataset")
print(str(np.sum(detailed_listings_back.isnull().sum()))+ " in current dataset")
print(str(np.sum(detailed_listings_back.isnull().sum())/detailed_listings_back.shape[0]/detailed_listings_back.shape[1]) + "% of dataset is null")

Dataset shape is (1497, 166)
25057 null cells in initial dataset
400 in current dataset
0.0016096449927968388% of dataset is null


In [67]:
detailed_listings_csv_data = detailed_listings_back.to_csv('l1_detailed_listings.csv', index = False) 

# Train/Test Split

In [68]:
from sklearn.model_selection import train_test_split
y = list(detailed_listings_back["rental_income"])
detailed_listings_back = detailed_listings_back.drop("rental_income", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(detailed_listings_back, y, test_size=0.2, random_state=0)


In [69]:
missing_col = ['check_in_time', 'check_out_time']
for i in missing_col:
    X_train.loc[X_train.loc[:,i].isnull(),i]=X_train.loc[:,i].median()
    X_test.loc[X_test.loc[:,i].isnull(),i]=X_test.loc[:,i].median()
#np.median(X_train['check_in_time'], axis = 0)

C:\Users\nates\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [70]:
from sklearn.impute import KNNImputer
bedcors = ["bedrooms","person_capacity", "beds", "bathrooms", "guests_included", "cleaning_fee_native"]
imputer = KNNImputer(n_neighbors=2)

df_bedcor = X_train[bedcors]
df_bedcor_post = imputer.fit_transform(df_bedcor)
X_train.bedrooms = df_bedcor_post[:,0]
X_train.cleaning_fee_native = df_bedcor_post[:,5]

df_bedcor = X_test[bedcors]
df_bedcor_post = imputer.fit_transform(df_bedcor)
X_test.bedrooms = df_bedcor_post[:,0]
X_test.cleaning_fee_native = df_bedcor_post[:,5]

C:\Users\nates\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\nates\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [71]:
seccors = ["security_deposit_native","min_nights","listing_weekend_price_native", "picture_count",
          "cleaning_fee_native", "listing_weekend_price_native"]

df_seccor = X_train[seccors]
df_seccor_post = imputer.fit_transform(df_seccor)
X_train.security_deposit_native = df_seccor_post[:,0]


df_seccor = X_test[seccors]
df_seccor_post = imputer.fit_transform(df_seccor)
X_test.security_deposit_native = df_seccor_post[:,0]

C:\Users\nates\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\nates\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [72]:
print(X_test.isnull().sum()/X_test.shape[0])
print(X_train.isnull().sum()/X_train.shape[0])

city                                          0.0
user_id                                       0.0
price                                         0.0
instant_bookable                              0.0
bedrooms                                      0.0
beds                                          0.0
bathrooms                                     0.0
min_nights                                    0.0
neighborhood                                  0.0
person_capacity                               0.0
zipcode                                       0.0
cancellation_policy                           0.0
property_type                                 0.0
reviews_count                                 0.0
picture_count                                 0.0
bed_type_category                             0.0
require_guest_profile_picture                 0.0
require_guest_phone_verification              0.0
check_in_time                                 0.0
check_out_time                                0.0


In [73]:
X_frames = [X_train, X_test] 
X_result = pd.concat(X_frames) 
X_result = X_result.sort_index()
X_result['rental_income'] = y
detailed_listings_csv_data = X_result.to_csv('l2_detailed_listings.csv', index = False) 

In [74]:
categorical_columns = ['city', 'instant_bookable', 'neighborhood',
                      'zipcode','property_type','cancellation_policy',
                      'bed_type_category', 'require_guest_profile_picture',
                      'require_guest_phone_verification', 
                      'locale', 'property_type_id', 'description_locale',
                      'is_location_exact', 'cancel_policy_short_str',
                      'instant_book_enabled', 'user_id']

In [75]:
y = list(X_result["rental_income"])
X_result = X_result.drop("rental_income", axis = 1)
X_result = pd.get_dummies(data=X_result,columns = categorical_columns)
X_train, X_test, y_train, y_test = train_test_split(X_result, y, test_size=0.2, random_state=0)


In [86]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet
rf = RandomForestRegressor(n_estimators = 100)
svr = SVR()
lassoCV = LassoCV(alphas=np.linspace(0.00001,1,1000), cv=10,tol = 0.00001,max_iter = 10000)

In [87]:
grid_search_lasso=lassoCV.fit(X_train, y_train)
print('train:',grid_search_lasso.score(X_train, y_train),'  test:',grid_search_lasso.score(X_test, y_test))

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 563704.1744656153, tolerance: 22798.324879777156
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 452365.7895659432, tolerance: 22798.324879777156
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 550029.9240184501, tolerance: 22798.324879777156
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 81689.06396261975, tolerance: 23063.456497623025
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 164293.74634497985, tolerance: 23063.456497623025
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2049207.730559349, tolerance: 23063.456497623025
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1313865.1487367973, tolerance: 22372.925801484234
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 470419.4042406492, tolerance: 22372.925801484234
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 365814.3892532252, tolerance: 22372.925801484234
  positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

train: 0.8983284674380545   test: 0.6900883320298908


In [89]:
svr_grid_param = [{'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
                   'degree': range(3),
                   'gamma': ['scale', 'auto'],
                   'shrinking':[True,False]}]
grid_search_svr = GridSearchCV(estimator=svr, param_grid=svr_grid_param, scoring='r2', cv=5, return_train_score=True)
grid_search_svr = grid_search_svr.fit(X_train, y_train)

In [ ]:
#James suggests running a couple between 100 and 1000 coursely to determine whether CV is necessary
grid_para_tree = [{
    "criterion": ["mse","mae"],
    "max_depth": range(1, 15, 4),
    "min_samples_split": range(2,30,5),
    "n_estimators": range(100,3000,100)
}]
#tree_model.set_params(random_state=34)
grid_search_rf = GridSearchCV(rf, grid_para_tree, cv=5, scoring='r2', n_jobs=-1)
grid_search_rf = grid_search_rf.fit(X_train, y_train)

In [ ]:
from mlxtend.regressor import StackingCVRegressor
stack = StackingCVRegressor(regressors=(lassoCV(alphas=np.linspace(0.00001,1,1000), cv=10,tol = 0.00001,max_iter = 10000), 
                            GridSearchCV(estimator=svr, param_grid=svr_grid_param, scoring='r2', cv=5, return_train_score=True),
                            GridSearchCV(rf, grid_para_tree, cv=5, scoring='r2', n_jobs=-1)),
                            meta_regressor=RandomForestRegressor(criterion = 'mae'), cv=10,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            random_state=1)

In [ ]:
stack.fit(X_train, y_train)
pred = stack.predict(X_test)
score = mean_absolute_error(y_test, pred)
print('Model: {0}, MAE: {1}'.format(type(stack).__name__, score))

In [78]:
rfe = RFE(estimator=rf, n_features_to_select=1, step=1)
rfe.fit(X_result, y)
ranking = rfe.ranking_.reshape(X_result[0].shape)

KeyError: 0